In [1]:
import gettext
import math
import random
import sys ## sleep 같은 것을 줄때
from time import sleep
import pygame
from pygame.locals import * 

##### 프로그램 전체 초기화 및 변수 설정###

pygame.init() ######### pygame 초기화

# 배경이미지의 크기가 1000 X 800 임
Window_Width = 1000
Window_Height = 600

BLACK = (0,0,0)
WHITE = (250,250,250)
YELLOW = (250,250,20)
BLUE = (20,20,250)

## 게임 틀 설정 ##
screen = pygame.display.set_mode((Window_Width, Window_Height))
pygame.display.set_caption('어느 평험한 대학생의 한학기') ## 게임 타이틀
pygame.display.set_icon(pygame.image.load('main_character.png')) ## 게임 아이콘
fps_clock = pygame.time.Clock() ## 게임 프레임
FPS = 60 ## 초당 60프레임
Academic_score = 0
Social_score = 0
total_bomb = 0

#### 주요 파일 불러오기/ 배경화면, 배경음, 효과음 ###
base_font = pygame.font.Font('Recipekorea_FONT.ttf', 30) ## 글폰트
background = pygame.image.load('background.jpg')  ## 게임 배경
Item_sound1 = pygame.mixer.Sound('item2_sound.wav') ## 아이템획득효과음
Item_sound2 = pygame.mixer.Sound('item2_sound.wav') ## 아이템획득효과음
bomb_sound = pygame.mixer.Sound('bomb_sound.wav') ## 과제폭탄 맞는 소리
bomb_effect = pygame.image.load('bomb2.png') ## 과제폭탄 맞을 때 효과 이미지
pygame.mixer.music.load('bright_house_music.mp3') ## 배경음 
back_rect = background.get_rect()
# 교수님이 뿌리시는 학점
score_A = pygame.image.load('A.png')
score_B = pygame.image.load('B.png')
score_C = pygame.image.load('C.png')
score_D = pygame.image.load('D.png')
score_F = pygame.image.load('F.png')




######################################################################
class Main_character(pygame.sprite.Sprite):
    def __init__(self, xpos ,ypos):
        super(Main_character, self).__init__() # 부모의 초기화를 오버라이딩한다.
        self.image = pygame.image.load('main_character.png')
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.centerx = self.rect.centerx
        self.centery = self.rect.centery
        # 주인공 이미지의 가로와 세로 길이
        self.size = self.image.get_rect().size
        self.width = self.size[0]
        self.height = self.size[1]
        
        
    # 경계밖으로 넘어가면 넘어가지 않도록 조정
    def set_pos(self, x,y):  #위치조정
        self.rect.x = x 
        if self.rect.x < 0 :
            self.rect.x = 0
        elif self.rect.x > Window_Width - self.width:
            self.rect.x = Window_Width - self.width
        self.rect.y = y 
        if self.rect.y < 0 :
            self.rect.y = 0
        elif self.rect.y > Window_Height - self.height:
            self.rect.y = Window_Height - self.height
        
    def collide(self, sprites): # 충돌판정
        for sprite in sprites: # sprites에 포함된 모든 객체들을 검사
            if pygame.sprite.collide_rect(self, sprite): # pygame의 충돌판정함수사용
                return sprite # 충돌된 sprite를 반환
######################################################################

## 화면의 상단 좌 우측에서 Bomb이 생성된다. ##
class Bomb(pygame.sprite.Sprite):
    def __init__(self, xpos, ypos, hspeed, vspeed):
        super(Bomb, self).__init__()  ## 부모 오버라이딩
        bomb = pygame.image.load('bomb.png')
        self.image = bomb
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.set_direction()
    
    # 과제폭탄이 날아가는 방향에 따라 이미지의 기울기가 달라진다.
    def set_direction(self):
        if self.vspeed > 0:
            if self.hspeed > 0:
                self.image = pygame.transform.rotate(self.image, 305)
            elif self.hspeed < 0:
                self.image = pygame.transform.rotate(self.image, 45)
        else:
            if self.hspeed > 0:
                self.image = pygame.transform.rotate(self.image, 270)
            elif self.hspeed < 0:
                self.image = pygame.transform.rotate(self.image, 90)
    
   # 폭탄이 움직이는 것을 갱신해주는 함수
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵 바깥으로 나갈 때
        if self.collide():
            self.kill() # 객체가 사라짐
            
    #맵 바깥으로 나가면 객체 삭제(bool)
    def collide(self):
        if self.rect.x < 0 - self.rect.height or self.rect.x > Window_Width:
            return True
        elif self.rect.y < 0 - self.rect.height or self.rect.y > Window_Height:
            return True
    
    def explosion(self):
        bomb_sound.play()
        screen.blit(bomb_effect, [self.rect.x , self.rect.y])
        self.kill()
            
#######################################################################
# 실질적으로 Bomb을 생성하는 부분. 방향과 속력을 결정한다.(랜덤으로)
def random_bomb(speed):
    random_direction = random.randint(1,3) # screen을 기준으로 3방향
    RandD = random.randint(1,3) # 기준선을 기준으로 세방향의 기울기
    
    #폭탄이 떨어지는 방향을 설정한다. 상, 좌 , 우에서 출현한다.
    #Rock(xpos, ypos, hspeed, vspeed)
    #위에서 아래로
    if random_direction == 1:
        if RandD == 1:
            return Bomb(random.randint(0, Window_Width-30), 0, 0, speed)
        elif RandD == 2:
            return Bomb(random.randint(0, Window_Width-30), 0, speed, speed)
        elif RandD == 3:
            return Bomb(random.randint(0, Window_Width-30), 0, -speed, speed)
        
    # 오른쪽에서 왼쪽
    elif random_direction == 2:
        if RandD == 1:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, 0)
        elif RandD == 2:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, speed)
        elif RandD == 3:
            return Bomb(Window_Width, random.randint(0, Window_Height-30), -speed, -speed)
    #왼쪽에서 오른쪽
    elif random_direction == 3:
        if RandD == 1:
            return Bomb(0, random.randint(0, Window_Width-30), speed, 0)
        elif RandD == 2:
            return Bomb(0, random.randint(0, Window_Width-30), speed, speed)
        elif RandD == 3:
            return Bomb(0, random.randint(0, Window_Width-30), speed, -speed)

#######################################################################
# 속도와 좌표를 설정해주면 친구 캐릭터가 화면을 떠다니게 된다. 
# 생성되고 벽과 5번 출돌하면 사라진다.
class Friend(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(Friend, self).__init__()
        self.image = pygame.image.load('friend.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 맵전체에서 랜덤생성
        self.rect.x = x - self.rect.centerx
        self.rect.y = y - self.rect.centery
        self.collide_count = 0 ##
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
            
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
############################################################################        
# A ~ F 객체 
class Score(pygame.sprite.Sprite):
    def __init__(self, x, y, hspeed, vspeed, score):
        super(Score, self).__init__()
        ## 스코어 선택
        if score == 0:
            self.image = pygame.image.load('A.png')
        elif score == 1:
            self.image = pygame.image.load('B.png')
        elif score == 2:
            self.image = pygame.image.load('C.png')
        elif score == 3:
            self.image = pygame.image.load('D.png')
        elif score == 4:
            self.image = pygame.image.load('F.png')
  
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        

        ## 생성위치
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 ##
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
            
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
############################################################################
# 총 3단계의 보스 교수님.. 시험을 보고 점수를 뿌리신다.
class Professor(pygame.sprite.Sprite):
    def __init__(self, x, y,hspeed, vspeed):
        super(Professor, self).__init__()
        self.image = pygame.image.load('professor.png')
        self.rect = self.image.get_rect()
        self.hspeed = hspeed
        self.vspeed = vspeed
        
        ## 생성위치
        self.rect.x = x
        self.rect.y = y
        self.collide_count = 0 ##
        
    def update(self):
        self.rect.x += self.hspeed
        self.rect.y += self.vspeed
        #  맵  경계부분에 닿았을때, 방향이 반대가 된다
        # 수직인 축의 속도는 그대로.
        col = self.collide()
        if col == 1:
            self.hspeed = -self.hspeed
        elif col == 2:
            self.vspeed = -self.vspeed
        # 생성되고 벽과 5번 출돌하면 사라진다.
        if self.collide_count >= 5:
            self.kill()
             
    #맵 경계에 닿는 판정// X Y 축을 판단한다.
    def collide(self):
        # 좌우 경계에 닿았을 때
        if self.rect.x < 0  or self.rect.x > Window_Width-self.rect.width:
            self.collide_count += 1 # 충돌카운트
            return 1
        # 상하 경계에 닿았을 때
        elif self.rect.y < 0  or self.rect.y > Window_Height- self.rect.height:
            self.collide_count += 1 # 충돌카운트
            return 2
    def Spread_Score(self, Hspeed, Vspeed):
        for i in range(20):
            
            Score(self.rect.centerx, self.rect.centery,Hspeed, Vspeed,  )
                


#######################################################################
## 텍스트를 그리기 위한 함수
def draw_text(text, font, surface, x, y, color):
    text_obj = font.render(text, True, color)
    text_rect = text_obj.get_rect()
    text_rect.centerx = x
    text_rect.centery = y
    surface.blit(text_obj, text_rect)
#  surface.blit() 메소드는 다른 Surface 객체를 자신에게 그려넣는 일을 한다. 
# 이 메소드의 1번째 매개변수는 그려넣을 Surface 객체, 
# 2번째 매개변수에 해당 Surface 객체를 그려넣을 좌표

## 폰트설정 ##
# font = pygame.font.Font('LexiGulim.ttf',30)  #폰트 설정
# text = font.render("글자출력",True,(28,0,0))  #텍스트가 표시된 Surface 를 만듬 (쓸 텍스트, 안티에일리어싱사용여부, 글자색)
# background.blit(text,(870,20))              #화면에 표시 surface 객체(text)를 background에 겹쳐서 표시


#########################################################################

def Run_game():
    
    global Academic_score, Social_score
    
    # 하루가 3초이며 3/2일 부터 6/20일까지의 시간을 그린다.
    # 따라서 시간은 111일로 333초가 플레이타임이다.
    time = 0 # 전체 시간
    frame_counter = 0
    day = 0 # 하루 = 3초
    
    pygame.mixer.music.play(-1)  # -1은 무한반복하라는 뜻/ 배경음 반복
    
    main_character = Main_character(Window_Width/2, Window_Height-80)
    # 파이썬은 그룹관리를 할 수 있으므로 객체를 만들어 사용한다
    bombs = pygame.sprite.Group() # 이것과 충돌 시 
    friends = pygame.sprite.Group()
    
    bomb_speed = 1
    min_bomb_speed = 1
    max_bomb_speed = 1
    occur_of_bombs = 1
    Friend_speed = 1
    occur_prob = 10
    total_bomb = 0
    
    main_X = 0
    main_Y = 0
    
    # 교우관계 - > 퍼센트게이지
    Academic_score = 0
    Social_score = 0
    # 이 게이지에 따라 소셜점수에 대한 획득량이 달라짐 낮으면 소셜점수가 떨어질 수 있음.
    Social_gauge = 50 # 초기 소셜 포인트 게이지 
    
    
    paused = False # 일시정지 bool
    while True:
        pygame.display.update()
        fps_clock.tick(FPS)  # 초당 프레임. ->60
        
        
        # 일시정지 시
        if paused:
            for event in pygame.event.get():
                # 다시 p가 눌리면 일시정지 해제
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_p:
                        paused = not paused
                    elif event.key == pygame.K_q:
                        return 'initial_screen'
                # 게임종료버튼 X를 누를 때,
                if event.type == pygame.QUIT:
                    return 'quit'
        else:
            frame_counter += 1 # 프레임 세기  -> 초를 셀때 사용
            default_speed = 3 # 기본속도
            
            # 배경 이미지를 프레임마다 업데이트
            if time <= 100:
                screen.blit(background, [0,0])
            elif time <= 200:
                screen.blit(background, [0,0])
            elif time <= 300:
                screen.blit(background, [0,0])
            else:
                screen.blit(background, [0,0])
            
            
            
            # 300초 때 한번에 생기는 bomb 갯수 증가!
            occur_of_bombs = 1 + int(time / 300) 
            # 100초마다 최고 속도 업!
            max_bomb_speed = default_speed + int(time/ 100)
            
            # 20분의 1의 확률로 1프레임마다 bomb이 생긴다.
            if random.randint(1, occur_prob*2) == 1:
                for i in range(occur_of_bombs):
                    bombs.add(random_bomb(random.randint(min_bomb_speed, max_bomb_speed)))
                    total_bomb += 1
                # 300분의 1확률로 1프레임마다 friend가 생긴다.
                if random.randint(1, occur_prob*1.5) == 1:
                    # 스피드가 0인 경우 1로 바꿔준다.
                    occur_hspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    occur_vspeed = random.randint(-max_bomb_speed, max_bomb_speed)
                    if occur_hspeed == 0:
                        occur_hspeed = 1
                    if occur_vspeed == 0:
                        occur_vspeed = 1
                    
                    friend = Friend(random.randint(70, Window_Width - 70),
                            random.randint(70, Window_Height - 70),
                            occur_hspeed,occur_vspeed)
                    friends.add(friend)
            # 1일 = 6초
            day = int(time) // 3
            # 1초마다 SP가 1씩떨어짐. 100초가 지날때마다 떨어지는 양이 많아진다.
            if frame_counter % 60 == 0: 
                Social_gauge -= (1 + int(time) // 100)
            
            draw_text('AP: {}'.format(Academic_score),base_font,screen,Window_Width*0.1 ,20, YELLOW)
            draw_text('day: {}'.format(day),base_font,screen, Window_Width/2 ,20, WHITE)
            time += 1/60
            draw_text('SP: {:.1f}%'.format(Social_gauge),base_font,screen,Window_Width *0.9 , 20, WHITE)
            draw_text('SP: {}'.format(Social_score),base_font,screen,Window_Width *0.9 , 60, WHITE)
            
            # 반복적으로 그려준다. 위치를 조정한다.
            # sprite를 그룹으로 사용시 항상 업데이트 해준다.
            bombs.update()
            friends.update()
            bombs.draw(screen)
            friends.draw(screen)
            
            # 충돌 판정 시 효과와 객체 삭제
            bomb = main_character.collide(bombs)
            friend = main_character.collide(friends)
            
            # bomb과 주인공이 충돌한 것이 있으면 (sprite가 반환되면)
            if bomb:
                bomb.explosion() # 폭발효과
                Academic_score -= 10
            # friend와 주인공이 충돌한 것이 있으면(sprite가 반환되면)
            elif friend:
                Social_gauge += 10
                friend.kill()
            # 주인공의 위치 업데이트
            screen.blit(main_character.image, main_character.rect)
  
            
            # 주인공의 위치를 방향키로 움직인다. 
            # 방향키를 누르고 있으면 그 쪽 방향으로 프레임마다 5씩 이동한다.
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT]:
                main_X = -5
            elif keys[pygame.K_RIGHT]:
                main_X = 5
            if keys[pygame.K_UP]:
                main_Y = -5
            elif keys[pygame.K_DOWN]:
                main_Y = 5
                
#                # 부드럽지 못한 움직임
#                 if event.type in [pygame.KEYDOWN]:
#                     if event.key == pygame.K_LEFT:
#                         main_X -= 5
#                     if event.key == pygame.K_RIGHT:
#                         main_X += 5
#                     if event.key == pygame.K_UP:
#                         main_Y -= 5
#                     if event.key == pygame.K_DOWN:
#                         main_Y += 5
        
            for event in pygame.event.get():
                # 키보드를 때면 멈춤
                if event.type in [pygame.KEYUP]:
                    if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                        main_X = 0
                    elif event.key == pygame.K_UP or event.key == pygame.K_DOWN :
                        main_Y = 0
                        
                # 일시 정지 버튼을 눌렀을 때
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_p:
                        paused = not paused #반대상태
                        # True이면
                        if paused:
                            # 일시정지시 불투명한 surface 가 화면을 채우도록 한다.
                            pause_surf = pygame.Surface((Window_Width, Window_Height))
                            pause_surf.set_alpha(100)
                            screen.blit(pause_surf, pause_surf.get_rect())
                            # 일시정지 텍스트를 화면에 띄운다.
                            draw_text('일시정지',
                                     pygame.font.Font('Recipekorea_FONT.ttf', 60),
                                     screen, Window_Width/ 2, Window_Height/2, WHITE)
                
                # close 버튼을 눌렀을 때,
                if event.type == pygame.QUIT: 
                    return 'quit'
                
            # 주인공의 움직임 조정 및 경계처리(맵 밖으로 못나가게)
            main_character.set_pos(main_character.rect.x+main_X,main_character.rect.y+main_Y)
#################################################            
            # 초당 소셜스코어 획득량
            if frame_counter % 60 == 0:
                if Social_gauge >= 100:
                    Social_score += 5
                elif Social_gauge >= 80:
                    Social_score += 3
                elif Social_gauge >= 60:
                    Social_score += 1
                elif Social_gauge >= 40:
                    Social_score -= 0
                elif Social_gauge >= 20:
                    Social_score -= 1
                elif Social_gauge >= 0:
                    Social_score -= 3
                else:
                    Social_score -= 5
                    
            
            
            
            # 시간이 모두 끝났을 때, 결과화면으로 가기
            if day >= 666 :
                return 'initial_screen'
            
    return 'initial_screen'

####################################################################
def initial_screen():
    # 초기화면 이미지
    #initial_img = pygame.image.load('')
    initial_img = background
    screen.blit(initial_img, [0, 0])
    
    draw_text('평범한 대학생으로 살기', pygame.font.Font('Recipekorea_FONT.ttf', 60),
              screen, Window_Width / 2, Window_Height / 3.5, YELLOW)
    draw_text("마우스 버튼 또는 'S'키를 눌러 게임을 시작하세요. ", pygame.font.Font('Recipekorea_FONT.ttf', 30),
              screen, Window_Width / 2, Window_Height /1.5 , YELLOW)
    draw_text("'Q'를 누르면 게임이 종료됩니다. ", pygame.font.Font('Recipekorea_FONT.ttf', 30),
              screen, Window_Width / 2, Window_Height / 1.2, YELLOW)
    
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                return 'quit'
            elif event.key == pygame.K_s:
                return 'play'
        if event.type == pygame.MOUSEBUTTONDOWN:
            return 'play'
        if event.type == pygame.QUIT:
            return 'quit'
        
    return 'initial_screen'



# 메인 시스템 루프
def main_loop():
    command = 'initial_screen'
    while command != 'quit':
        if command == 'initial_screen':
            command = initial_screen()
        elif command == 'play':
            command = Run_game()
    
    pygame.quit()
    
    
main_loop()
        






pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
